In [1]:
import rinna_utils


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home1/tanimoto.j/anaconda3/envs/llm/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda114.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 114
CUDA SETUP: Loading binary /home1/tanimoto.j/anaconda3/envs/llm/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda114.so...


/home1/tanimoto.j/anaconda3/envs/llm/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home1/tanimoto.j/anaconda3/envs/llm did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home1/tanimoto.j/anaconda3/envs/llm/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('[%y/%m/%d %T] ')}
  warn(msg)
/home1/tanimoto.j/anaconda3/envs/llm/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)
/home1/tanimoto.j/anaconda3/envs/llm/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] 

In [2]:
ls ../models

lora-rinna-3.6b/  rinna3.6b/


In [17]:
models_path = '../models'

In [18]:
model = rinna_utils.load_model(f'{models_path}/rinna3.6b')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [27]:
tokenizer = rinna_utils.load_tokenizer(f'{models_path}/rinna3.6b')

In [20]:
roles = ['ユーザー: ', 'システム: ']

In [21]:
prompt_list = ['会話をしましょう。まずはあなたの自己紹介をお願いします。設定は以下のとおりです。「大学院生, 情報工学専攻, 画像処理が専門」']

In [22]:
prompt = [f"{roles[(i)%2]}{p}" for i, p in enumerate(prompt_list)]
prompt:str = '<NL>'.join(prompt)
prompt += '<NL>システム: '

In [23]:
print(rinna_utils.generate(prompt, tokenizer, model, max_new_tokens=150, output_only_generated=False, escape_special_token=False))

ユーザー: 会話をしましょう。まずはあなたの自己紹介をお願いします。設定は以下のとおりです。「大学院生, 情報工学専攻, 画像処理が専門」<NL>システム: 大学院の学部生です。 趣味は、美術鑑賞、写真撮影です. これらの趣味を共有できる人を探しています。 Muse artistに入会したいですが、まだ入会したばかりで、しばらくは入会金欠ですので. Musartista入会金を貯めつつ、入会を検討しています. (注:入会して、金が貯まったら、すぐに入会する予定です). TLIST workers入会しました。入会が待ち遠ざかります。 TSLISTの入会は、とても楽しいです! :) 注) TLSIST入会時、私は学部でした. SLS 入会時に学部を中退しましたが、今は卒業しまして再


In [24]:
exit_code = ['exit', 'quit']

In [25]:
def gen_prompt(prompt):
    prompt:str = '<NL>'.join(prompt)
    prompt += '<NL>システム: '
    return prompt

In [28]:
def gen(prompt, model, tokenizer):
    input_ids = rinna_utils.tokenize(prompt, tokenizer)['input_ids'].cuda()
    input_ids = input_ids[:, :-1]
    input_token_len = input_ids.shape[1]

    # https://huggingface.co/docs/transformers/v4.31.0/en/main_classes/text_generation#transformers.GenerationConfig
    outputs = model.generate(
        input_ids=input_ids,
        max_new_tokens=150,
        do_sample=False,
        num_beams=3,
        temperature=1,
        no_repeat_ngram_size=2,
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id
    )

    outputs = outputs[0]
    outputs = outputs[input_token_len:]
    outputs = outputs[outputs != tokenizer.bos_token_id]
    outputs = outputs[outputs != tokenizer.eos_token_id]

    text: str = tokenizer.decode(outputs.tolist())

    text = text.replace("<NL>", '\n')
    return text

prompt = []
while True:
    print('----')
    input_text = input('ユーザー > ')
    if input_text in exit_code:
        break
    prompt.append(f'ユーザー: {input_text}')
    output_text =gen(
        gen_prompt(prompt), 
        model,
        tokenizer)
    print(f"システム > {output_text}")
    prompt.append(f'システム: {output_text}')

----
ユーザー > 遊びましょう。何したいですか？
システム > ボードゲームやカードゲームをしたいです。チェスやチェスセット、トランプなどがあります。また、カードやフィギュアのコレクションも持っています。それらを一緒に遊ぶことができます。ゲームのルールや遊び方を説明して、一緒に遊んでくれる人を募集しています。興味があれば、メッセージをお送りください。よろしくお願いします。
----
ユーザー > ボードゲームはどんなものができますか？
システム > チェス、将棋、囲碁、チェッカー、ドミノなど、多くの人気のあるゲームがあり、様々な難易度や難易度でプレイすることができ、家族や友人と遊ぶのに最適なゲームです! これらのゲームには、ボードや駒、ルールブックなどが付属しており、初心者から上級者まで幅広く楽しめるようになっており、プレイの仕方やルールの説明も丁寧に行っていきますので、お気軽にご参加ください!
----
ユーザー > チェスをやってみたいです。
システム > はい、チェスは人気の高いゲームの一つです。初心者や中級者に適したゲームであり、戦略や戦術を学ぶのにも役立ち、友達や家族と一緒にプレイするのも楽しいですよ。 駒や盤、駒の置き方などの基本的なルールから始め、練習を重ねることで上達することができます!
----
ユーザー > 盤面を表示してください
システム > 素晴らしいですね。盤面の画像を用意しました。駒の動きや戦略の重要な部分を説明し、ゲームの流れを簡単に理解できるようにしています:) ゲームに慣れてきたら、自分で駒を配置したり、相手の動きを観察して戦略を練ったりしてみましょう!
----
ユーザー > 画像で表示しないでください。アスキーアートのように、マス目を文字で書いていい感じに表示してください。
システム > ああ、分かりました。マス目に文字を書くのは難しいですが、今後の改善に活かします!チェスのゲームで重要な要素や重要な手筋を重点的に説明するように心がけ、遊びやすく楽しいゲームになるように頑張りますね!
----
ユーザー > チェスはやめます
システム > わかりました、ごめんなさい。今後チェスをするかどうかは、自分自身で決める必要がありますね、良い選択をされることを願ってます:-)
----
ユーザー > クイズをしましょう。問題をだして

In [29]:
prompt

['ユーザー: 遊びましょう。何したいですか？',
 'システム: ボードゲームやカードゲームをしたいです。チェスやチェスセット、トランプなどがあります。また、カードやフィギュアのコレクションも持っています。それらを一緒に遊ぶことができます。ゲームのルールや遊び方を説明して、一緒に遊んでくれる人を募集しています。興味があれば、メッセージをお送りください。よろしくお願いします。',
 'ユーザー: ボードゲームはどんなものができますか？',
 'システム: チェス、将棋、囲碁、チェッカー、ドミノなど、多くの人気のあるゲームがあり、様々な難易度や難易度でプレイすることができ、家族や友人と遊ぶのに最適なゲームです! これらのゲームには、ボードや駒、ルールブックなどが付属しており、初心者から上級者まで幅広く楽しめるようになっており、プレイの仕方やルールの説明も丁寧に行っていきますので、お気軽にご参加ください!',
 'ユーザー: チェスをやってみたいです。',
 'システム: はい、チェスは人気の高いゲームの一つです。初心者や中級者に適したゲームであり、戦略や戦術を学ぶのにも役立ち、友達や家族と一緒にプレイするのも楽しいですよ。 駒や盤、駒の置き方などの基本的なルールから始め、練習を重ねることで上達することができます!',
 'ユーザー: 盤面を表示してください',
 'システム: 素晴らしいですね。盤面の画像を用意しました。駒の動きや戦略の重要な部分を説明し、ゲームの流れを簡単に理解できるようにしています:) ゲームに慣れてきたら、自分で駒を配置したり、相手の動きを観察して戦略を練ったりしてみましょう!',
 'ユーザー: 画像で表示しないでください。アスキーアートのように、マス目を文字で書いていい感じに表示してください。',
 'システム: ああ、分かりました。マス目に文字を書くのは難しいですが、今後の改善に活かします!チェスのゲームで重要な要素や重要な手筋を重点的に説明するように心がけ、遊びやすく楽しいゲームになるように頑張りますね!',
 'ユーザー: チェスはやめます',
 'システム: わかりました、ごめんなさい。今後チェスをするかどうかは、自分自身で決める必要がありますね、良い選択をされることを願ってます:-)',
 'ユーザー: クイズをしましょう。問題をだ